In [1]:
import boto3
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [2]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'lambda/scrap_idealista')

import scrapper

items = scrapper.scrap()

ModuleNotFoundError: No module named 'scrapper'

In [ ]:
def add_updated_columns(items):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('scrapped_ads')

    response = table.scan(
        AttributesToGet=['id','price', 'created','price_update','date_update'],
        ScanFilter={'id': {'AttributeValueList': [it['id'] for it in items],'ComparisonOperator': 'IN'}}
    )
    scanned_items = response['Items']

    scanned_items[0]['price'] -= 5
    scanned_items[20]['price'] -= 5
    #array to hash to be faster to look
    old_items_map = { }
    for it in scanned_items:
        if 'price_update' not in it['id']:
            it['price_update'] = [it['price']]
        if 'date_update' not in it['id']:
            it['date_update'] = [it['created']]
        old_items_map[it['id']] = it

    # rows with new price get and updated array
    for it in items:
        if it['id'] in old_items_map and it['price'] != old_items_map[it['id']]['price']:
            old_items_map[it['id']]['price_update'].append(it['price'])
            old_items_map[it['id']]['date_update'].append(it['created'])
            it['price_update'] = old_items_map[it['id']]['price_update']
            it['date_update'] = old_items_map[it['id']]['date_update']
            print('Price update!\n\t{}\n\t{}'.format(it, old_items_map[it['id']]))
add_updated_columns(items)

In [ ]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('scrapped_ads')


In [ ]:
response = table.scan()
response

In [ ]:
len(response['Items'])

# filter ads with no geo
ad_with_geo = list(filter(lambda x: (('geo' in x) and (len(x['geo']) > 0)), response['Items']))

print('Items scanned:{:>5}\nItems with geo:{:>4}'.format(len(response['Items']), len(ad_with_geo)))

In [ ]:
ad_with_geo[0]['geo'][0]

for ad in ad_with_geo:
    geo = ad['geo'][0]
    ad['latitude'] = geo['latitude']
    ad['longitude'] = geo['longitude']
    ad['locality'] = geo['locality']
    ad['neighbourhood'] = geo['neighbourhood']
    ad['geo_name'] = geo['name']

In [ ]:
df = pd.DataFrame.from_dict(ad_with_geo)
df.describe()

In [ ]:
df_group_locality = df[df.locality.notnull()].groupby('locality')

data = []
for name, group in df_group_locality:
    subplot_id = "mapbox" + name
    trace = go.Scattermapbox(
        name=name,
        showlegend=True,
        lat=df[df.locality == name].latitude,
        lon=df[df.locality == name].longitude,
        mode="markers",
        marker=dict(size=14),
        text=name,
        subplot="mapbox",
    )
    # fig.add_trace(trace)
    data.append(trace)


In [ ]:
# ---------------------------
# Plot Color ad per municipality
# ---------------------------

layout = go.Layout(
    autosize=True,
    height=700,
    title="Color ad by area",
    # grid={"rows": rows, "columns": cols, "xgap": 0.1, "ygap": 0.2},
)


map_center = go.layout.mapbox.Center(
    lat=df["latitude"].mean(), lon=df["longitude"].mean()
)
layout["mapbox"] = dict(style="carto-positron", center=map_center, zoom=11,)

fig = go.Figure(data=data, layout=layout)
fig

In [ ]:
import urllib
#df.address = df.address.apply(urllib.parse.unquote)
#df[df.address.str.contains('Flat|flat')][['address','geo']]
df.head()